In [2]:
import os
import cv2
import torch
import numpy as np
import pandas as pd

import wandb
import datetime
from tqdm.auto import tqdm

from timm.models import resnet18, resnet101c, efficientnetv2_s, efficientnetv2_m, vit_tiny_r_s16_p8_384, swinv2_cr_small_384, swinv2_cr_tiny_384
from torchsummary import summary

In [3]:
model = vit_tiny_r_s16_p8_384(pretrained=False).cuda()
summary(model, (3, 384, 384))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
     StdConv2dSame-1         [-1, 64, 192, 192]           9,408
          Identity-2         [-1, 64, 192, 192]               0
              ReLU-3         [-1, 64, 192, 192]               0
      GroupNormAct-4         [-1, 64, 192, 192]             128
     MaxPool2dSame-5           [-1, 64, 96, 96]               0
            Conv2d-6          [-1, 192, 12, 12]         786,624
       HybridEmbed-7             [-1, 144, 192]               0
           Dropout-8             [-1, 145, 192]               0
          Identity-9             [-1, 145, 192]               0
         Identity-10             [-1, 145, 192]               0
        LayerNorm-11             [-1, 145, 192]             384
           Linear-12             [-1, 145, 576]         111,168
         Identity-13           [-1, 3, 145, 64]               0
         Identity-14           [-1, 3, 

In [ ]:
model

In [ ]:
model = resnet101c().cuda()
summary(model, (3, 224, 224))

In [ ]:
model = efficientnetv2_s().cuda()
summary(model, (3, 224, 224))

In [ ]:
model = efficientnetv2_m().cuda()
summary(model, (3, 224, 224))

In [ ]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from dataset import *

In [ ]:
tf = A.Compose([
    A.Resize(224, 224),
    A.Normalize(),
])

train_dataset = CustomDataset(
            "/opt/ml/level3_cv_finalproject-cv-01/model", is_train=True, tf=tf
        )

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
            dataset=train_dataset,
            batch_size=4,
            shuffle=True,
            num_workers=8,
            drop_last=True,
        )

In [ ]:
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.nn as nn
from sklearn.metrics import f1_score, precision_score


cls_criterion = nn.CrossEntropyLoss()
rcp_criterion = nn.CrossEntropyLoss()

with torch.no_grad():
    for images, labels, ingredients in train_loader:
        images, labels, ingredients = (
                images.clone().detach().cuda(),
                labels.clone().detach().cuda(),
                ingredients.clone().detach().cuda(),
            )
        print(f"image : {images.shape}")
        print(f"label : {labels.shape}")
        print(f"ingredient : {ingredients.shape}")
        print(f"max : {torch.max(images)}, min : {torch.min(images)}")

        cls_output, rcp_output = model(images)
        print(f"cls_output : {cls_output.shape}")
        print(f"rcp_output : {rcp_output.shape}")
        print(f"max : {torch.max(cls_output)}, min : {torch.min(cls_output)}")
        print(f"max : {torch.max(rcp_output)}, min : {torch.min(rcp_output)}")

        cls_output = F.sigmoid(cls_output)
        rcp_output = F.sigmoid(rcp_output)
        print(f"max : {torch.max(cls_output)}, min : {torch.min(cls_output)}")
        print(f"max : {torch.max(rcp_output)}, min : {torch.min(rcp_output)}")
        print(f"max : {torch.max(labels)}, min : {torch.min(labels)}")
        print(f"max : {torch.max(ingredients)}, min : {torch.min(ingredients)}")

        cls_loss = cls_criterion(cls_output, labels)
        rcp_loss = rcp_criterion(rcp_output, ingredients)
        print(f"Loss : {cls_loss}, {rcp_loss}")

        cls_thr = 0.5
        rcp_thr = 0.5
        cls_output = (cls_output >= cls_thr).float()
        rcp_output = (rcp_output >= cls_thr).float()
        cls_loss = cls_criterion(cls_output, labels)
        rcp_loss = rcp_criterion(rcp_output, ingredients)
        print(f"Loss : {cls_loss}, {rcp_loss}")

        cls_f1 = f1_score(y_pred=cls_output.cpu(), y_true=labels.cpu(), average="macro", zero_division=0)
        rcp_f1 = f1_score(y_pred=rcp_output.cpu(), y_true=ingredients.cpu(), average="macro", zero_division=0)
        cls_precision = precision_score(y_pred=cls_output.cpu(), y_true=labels.cpu(), average="macro", zero_division=0)
        rcp_precision = precision_score(y_pred=rcp_output.cpu(), y_true=ingredients.cpu(), average="macro", zero_division=0)
        print(f"F1-Score : {cls_f1}, {rcp_f1}")
        print(f"Precision Score : {cls_precision}, {rcp_precision}")
        raise
        for i in range(images.shape[0]):
            image = images[i].cpu()
            image = image.permute(1, 2, 0)
            image_np = image.numpy()
            print(train_dataset.classes[labels[i].item()])
            plt.imshow(image_np)
            plt.axis('off')
            plt.show()
        raise

In [ ]:
import torch
import matplotlib.pyplot as plt
from torch import cuda
from torch.optim.lr_scheduler import CosineAnnealingLR

In [ ]:
model.to("cuda" if cuda.is_available() else "cpu")
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
max_epoch = 50
scheduler = CosineAnnealingLR(optimizer, T_max=max_epoch, eta_min=1e-3)

In [ ]:
lrs = []
for epoch in range(max_epoch):
    optimizer.zero_grad()
    optimizer.step()
    scheduler.step()
    current_lr = optimizer.param_groups[0]["lr"]
    lrs.append(current_lr)
fig, ax = plt.subplots()
ax.plot(lrs)
plt.grid()
plt.show()

In [ ]:
import json
import os

path = "/opt/ml/level3_cv_finalproject-cv-01/model/data/json/milfeuille"
file_names = os.listdir(path)
for file_name in file_names:
    # new_name = file_name.replace("onirigi", "onigiri")
    with open(os.path.join(path, file_name)) as file:
        data = json.load(file)
    data['food_class'] = "milfeuille"
    with open(os.path.join("/opt/ml/level3_cv_finalproject-cv-01/model/milfeuille", file_name), "w") as file:
        json.dump(data, file)
    